In [1]:
import scipy as sp
import numpy as np
import pandas as pd
import timeit
import re
import json
import pickle
import fastparquet
import os
os.chdir('/mnt/t48/bighomes-active/sfeng/patentdiffusion/')
seed = 3
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
import datetime
import logging
import itertools

In [2]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO)
logger.addHandler(logging.FileHandler('Logs/pc_sim_{0}.log'.format(datetime.datetime.now().\
                                                            strftime("%Y-%m-%d"), 'a')))
print = logging.info
print('good day to you madam fiona')
print('started')
print(datetime.datetime.now())

INFO:root:good day to you madam fiona
INFO:root:started
INFO:root:2018-10-05 11:42:39.153621


In [ ]:
# Load patents
pdf = fastparquet.ParquetFile("RawData/Cleaned/patent_loc_unique_us_0628.parq")\
.to_pandas(["patent", "primclass", "inv_msa", "gyear"])
dup_pats = pd.read_pickle("RawData/Cleaned/duplicate_pattext_0712.pkl").tolist()
# Get relevant US Patents
pdf = pdf.loc[~pdf["patent"].isin(dup_pats)]
# Get rid of missing observations
pdf = pdf.dropna(how="any")
# Only use patents in patdict
pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
pat_dict = dict(zip(pat_dict, range(len(pat_dict))))
# Get row values
pdf["pat_row"] = pdf["patent"].map(pat_dict)
pdf = pdf.loc[pdf["pat_row"].notnull()]

### 1. Create patent sample dictionary for each year, PC, PC MSA

In [51]:
pc_dict = {}
pc_msa_dict = {}
max_size = 1000
for yr in range(1980, 2016):
    print(yr)
    print(datetime.datetime.now())
    # Control patent by assignee, primclass
    p2 = pdf.loc[(pdf["gyear"].isin(range(yr-5,yr)))]
    
    # Primary class patents
    pcd = p2.groupby(["primclass"])
    pcd = {(n,yr): g["pat_row"].tolist() if (len(g) < max_size) else np.random.choice(g["pat_row"].tolist(), 
            size=max_size, replace=False) for n,g in pcd}
    pc_dict.update(pcd)
    del(pcd)
    
    # PC MSA patents
    pcmd = p2.groupby(["primclass", "inv_msa"])
    pcmd = {n+(yr,): g["pat_row"].tolist() if (len(g) < max_size) else np.random.choice(g["pat_row"].tolist(), 
            size=max_size, replace=False) for n,g in pcmd}
    pc_msa_dict.update(pcmd)
    del(pcmd)

1980
2018-09-30 12:47:36.005610
1981
2018-09-30 12:47:46.764223
1982
2018-09-30 12:47:56.836563
1983
2018-09-30 12:48:16.434095
1984
2018-09-30 12:48:27.265533
1985
2018-09-30 12:48:38.474512
1986
2018-09-30 12:48:49.614143
1987
2018-09-30 12:49:00.140769
1988
2018-09-30 12:49:19.225333
1989
2018-09-30 12:49:30.598451
1990
2018-09-30 12:49:43.313347
1991
2018-09-30 12:49:55.128788
1992
2018-09-30 12:50:07.011631
1993
2018-09-30 12:50:20.170125
1994
2018-09-30 12:50:43.531859
1995
2018-09-30 12:50:56.786097
1996
2018-09-30 12:51:10.384832
1997
2018-09-30 12:51:25.016639
1998
2018-09-30 12:51:40.353776
1999
2018-09-30 12:52:03.907591
2000
2018-09-30 12:52:20.396601
2001
2018-09-30 12:52:36.581704
2002
2018-09-30 12:52:52.349374
2003
2018-09-30 12:53:08.518595
2004
2018-09-30 12:53:27.660108
2005
2018-09-30 12:53:45.881705
2006
2018-09-30 12:54:12.277115
2007
2018-09-30 12:54:30.268430
2008
2018-09-30 12:54:46.772478
2009
2018-09-30 12:55:00.168831
2010
2018-09-30 12:55:14.563335
2011
201

In [52]:
pickle.dump(pc_dict, open("DataStore/2018-10/pc_year_pat_row_dict_0930.pkl", "wb"))
pickle.dump(pc_msa_dict, open("DataStore/2018-10/pc_msa_year_pat_row_dict_0930.pkl", "wb"))

### 2. Create list of PC, PC MSA pairs to get sample from

In [3]:
pc_dict = pickle.load(open("DataStore/2018-10/pc_year_pat_row_dict_0930.pkl", "rb"))
pc_msa_dict = pickle.load(open("DataStore/2018-10/pc_msa_year_pat_row_dict_0930.pkl", "rb"))

In [4]:
files = {"naics_name": "DataStore/2018-08/Reg0919/naics_name_all_0824.parq",
         "primclass": "DataStore/2018-08/Reg0919/primclass_all_0824.parq",
}

ps = pd.DataFrame()
for k,v in files.items():
    l = fastparquet.ParquetFile(v).to_pandas(["tp_primclass", "tp_inv_msa", "tp_gyear",
                                              "op_primclass", "op_inv_msa"])
    ps = ps.append(l, ignore_index=True)
# Drop duplicates
print(len(ps))
ps = ps.drop_duplicates()
print(len(ps))

INFO:root:2868017
INFO:root:2194513


In [5]:
ps.head()

,tp_primclass,tp_inv_msa,tp_gyear,op_primclass,op_inv_msa
0,296.0,"Clinton Township, MI",2000,296.0,"Clinton Township, MI"
1,62.0,"Buffalo-Niagara Falls, NY",2001,101.0,"Buffalo-Niagara Falls, NY"
2,347.0,"San Diego-Carlsbad-San Marcos, CA",2001,726.0,"San Francisco-Oakland-Fremont, CA"
3,370.0,"New York-Northern New Jersey-Long Island, NY-N...",1996,372.0,"New York-Northern New Jersey-Long Island, NY-N..."
4,137.0,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",1991,269.0,"Portland-Vancouver-Hillsboro, OR-WA"


### 3. Similarity by PC pair, PC MSA pair

#### 3.1 PC Pair

Use only lower diagonal values of similarity matrix because the diagonals all equal 1, and only use similarities not equal to 0 (hopefully will be all the upper triangular zeroes, and few actual similarities) and less than 1. 

In [8]:
pc_dict = {k: np.array(v, dtype=int) for k,v in pc_dict.items() if len(v) > 0}
pc_msa_dict = {k: np.array(v, dtype=int) for k,v in pc_msa_dict.items() if len(v) > 0}

In [ ]:
import scipy.spatial.distance as distance
from sklearn.metrics.pairwise import cosine_similarity
dms = ["ldavecs", "docvecs"]

print("Loading pat dict")
print(datetime.datetime.now())
pat_dict = fastparquet.ParquetFile("RawData/Cleaned/patabs7615_us_no_dup.parq").to_pandas(["patent"])["patent"].tolist()
pat_dict = dict(zip(pat_dict, range(len(pat_dict))))

l1 = ps.copy()
for dm in dms:
    print((dm,"started"))
    print("Loading matrix and dict")
    print(datetime.datetime.now())

    pm = fastparquet.ParquetFile("DataStore/2018-07-P2/ML/{0}_pats_0712.parq".format(dm))\
    .to_pandas().values
    
    # 1. Similarity between primary classes
    
    # Drop duplicate primary class year pairs
    l2 = ps[["tp_primclass", "op_primclass", "tp_gyear"]].copy().drop_duplicates()
    l2["tp_pc_ind"] = list(zip(l2["tp_primclass"], l2["tp_gyear"]))
    l2["op_pc_ind"] = list(zip(l2["op_primclass"], l2["tp_gyear"]))
    # Only get values in dictionary
    l2 = l2.loc[l2["tp_pc_ind"].isin(pc_dict.keys()) & l2["op_pc_ind"].isin(pc_dict.keys())]
    
    print("Getting distance")
    print(datetime.datetime.now())
    iter_mat = iter(((pm[pc_dict[i]], pm[pc_dict[j]]) for i,j in zip(l2["tp_pc_ind"], l2["op_pc_ind"])))
    
    dis = np.zeros(len(l2))
    sd = np.zeros(len(l2))
    for n,i in enumerate(iter_mat):
        s = np.tril(cosine_similarity(i[0],i[1]),0)
        s = s[(s != 0) & (s < 1)]
        dis[n] = np.mean(s)
        sd[n] = np.std(s)
    
    l2["mean_sim_{0}_pc".format(dm)] = dis
    l2["sd_sim_{0}_pc".format(dm)] = sd
    del(dis, sd)
    
    # Normalize values
    l2["norm_mean_sim_{0}_pc".format(dm)] = np.nan
    l2.loc[l2["mean_sim_{0}_pc".format(dm)].notnull(), "norm_mean_sim_{0}_pc".format(dm)] =\
    scaler.fit_transform(l2.loc[l2["mean_sim_{0}_pc".format(dm)].notnull(), "mean_sim_{0}_pc".format(dm)]\
                         .values.reshape(-1,1))
    
    # Merge with original data
    print(len(l1))
    l1 = l1.merge(l2, how="left", on = ["tp_primclass", "op_primclass", "tp_gyear"])
    l1.to_pickle("DataStore/2018-10/temp_sim_save.pkl")
    print(len(l1))
    del(l2)
    print("Finished PC")
    print(datetime.datetime.now())
    
    # 2. Similarity between PC-MSA
    
    # Drop duplicates
    l2 = ps.copy().drop_duplicates()
    l2["tp_pc_m_ind"] = list(zip(l2["tp_primclass"], l2["tp_inv_msa"], l2["tp_gyear"]))
    l2["op_pc_m_ind"] = list(zip(l2["op_primclass"], l2["op_inv_msa"], l2["tp_gyear"]))
    # Only get values in dictionary
    l2 = l2.loc[l2["tp_pc_m_ind"].isin(pc_msa_dict.keys()) & l2["op_pc_m_ind"].isin(pc_msa_dict.keys())]
    
    print("Getting distance")
    print(datetime.datetime.now())
    
    iter_mat = iter(((pm[pc_msa_dict[i]], pm[pc_msa_dict[j]]) for i,j in zip(l2["tp_pc_m_ind"], l2["op_pc_m_ind"])))
    
    dis = np.zeros(len(l2))
    sd = np.zeros(len(l2))
    for n,i in enumerate(iter_mat):
        s = np.tril(cosine_similarity(i[0],i[1]),0)
        s = s[(s != 0) & (s < 1)]
        dis[n] = np.mean(s)
        sd[n] = np.std(s)
        
    l2["mean_sim_{0}_pc_msa".format(dm)] = dis
    l2["sd_sim_{0}_pc_msa".format(dm)] = sd
    del(dis, sd)
    
    # Normalize values
    l2["norm_mean_sim_{0}_pc_msa".format(dm)] = np.nan
    l2.loc[l2["mean_sim_{0}_pc_msa".format(dm)].notnull(), "norm_mean_sim_{0}_pc_msa".format(dm)] =\
    scaler.fit_transform(l2.loc[l2["mean_sim_{0}_pc_msa".format(dm)].notnull(), "mean_sim_{0}_pc_msa".format(dm)]\
                         .values.reshape(-1,1))
    
    print(len(l1))
    l1 = l1.merge(l2, how="left", on = ["tp_primclass", "tp_inv_msa", "op_primclass", "op_inv_msa", "tp_gyear"])
    l1.to_pickle("DataStore/2018-10/temp_sim_save.pkl")
    print(len(l1))
    del(l2)
    
    print("Finished PC MSA")
    print(datetime.datetime.now())

INFO:root:Loading pat dict
INFO:root:2018-10-05 13:11:31.599093
INFO:root:('ldavecs', 'started')
INFO:root:Loading matrix and dict
INFO:root:2018-10-05 13:11:33.714288
INFO:root:Getting distance
INFO:root:2018-10-05 13:11:54.511321
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:135: RuntimeWarning: Degrees of freedom <= 0 for slice
  keepdims=keepdims)
/homedir/eco/sfeng/bigdata/python/miniconda3/lib/python3.6/site-packages/numpy/core/_methods.py:105: RuntimeWarning: invalid value encountered in true_divide
  arrmean, rcount, out=arrmean, casting='unsafe', subok=False)
/homedir/eco/sf

In [15]:
ps = l1[['tp_primclass', 'tp_inv_msa', 'tp_gyear', 'op_primclass', 'op_inv_msa',
        "mean_sim_docvecs_pc", "mean_sim_ldavecs_pc", "mean_sim_docvecs_pc_msa", "mean_sim_ldavecs_pc_msa",
         "norm_mean_sim_docvecs_pc", "norm_mean_sim_ldavecs_pc", "norm_mean_sim_docvecs_pc_msa",
         "norm_mean_sim_ldavecs_pc_msa",
        "sd_sim_docvecs_pc", "sd_sim_ldavecs_pc", "sd_sim_docvecs_pc_msa", "sd_sim_ldavecs_pc_msa"]]
fastparquet.write("DataStore/2018-10/pc_sim_pairs_0929.parq", ps, compression="GZIP")

______________
Because I was stupid

In [86]:
l2 = l1.copy()
dup = l2.columns.get_duplicates()
cols = pd.Series(l2.columns)
for d in dup:
    cols.loc[cols==d] = [d+"_{0}".format(i) for i in range(len(cols.loc[cols==d]))]
l2.columns = list(cols)

In [87]:
l2 = l2[['tp_primclass', 'tp_inv_msa', 'tp_gyear', 'op_primclass', 'op_inv_msa',
"mean_sim_docvecs_pc_x_1", "sd_sim_docvecs_pc_x_1", "mean_sim_ldavecs_pc_x_1", "sd_sim_ldavecs_pc_x_1",
"mean_sim_docvecs_pc_msa", "sd_sim_docvecs_pc_msa", "mean_sim_ldavecs_pc_msa", "sd_sim_ldavecs_pc_msa"]]
l2.columns = ['tp_primclass', 'tp_inv_msa', 'tp_gyear', 'op_primclass', 'op_inv_msa',
"mean_sim_docvecs_pc", "sd_sim_docvecs_pc", "mean_sim_ldavecs_pc", "sd_sim_ldavecs_pc",
"mean_sim_docvecs_pc_msa", "sd_sim_docvecs_pc_msa", "mean_sim_ldavecs_pc_msa", "sd_sim_ldavecs_pc_msa"]

In [88]:
ps = l2[['tp_primclass', 'tp_inv_msa', 'tp_gyear', 'op_primclass', 'op_inv_msa',
        "mean_sim_docvecs_pc", "mean_sim_ldavecs_pc", "mean_sim_docvecs_pc_msa", "mean_sim_ldavecs_pc_msa",
        "sd_sim_docvecs_pc", "sd_sim_ldavecs_pc", "sd_sim_docvecs_pc_msa", "sd_sim_ldavecs_pc_msa"]]
fastparquet.write("DataStore/2018-10/pc_sim_pairs_0929.parq", ps, compression="GZIP")